# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Based from the goal we can tell the problem we are trying to solve, it leans to classification - supervised learning. The reason we can pick classification because the output consists of category. I recall in classification there is attributes and properties. Regression problem most of the time are continous output in the problem. Classification can be do the students require early intervention ? T/F. This can be first properties, second do the students not require early intervention ? T/F.

We need to give a students a label: needs early intervention or doesn't need early intervention. Those problems in which there are a finite number of possible outputs and they cannot be ordered are classification problems. Regression problems involve continuous outputs, or, at least, ordinal outputs. By start asking this question, we conclude that the output is categorical and continuous input.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [4]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
# should not include our target column, thus do substract by one
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = np.sum(student_data['passed'] == 'yes')

# TODO: Calculate failing students
n_failed = np.sum(student_data['passed'] == 'no')

# TODO: Calculate graduation rate
grad_rate = 100 * float (n_passed) / float(n_students)

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [5]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [15]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [16]:
# Imbalanced proportion issue, then we should benchmark our models training by use the score from predicting all yes values
from sklearn.metrics import f1_score
print ("\nF1 score for all yes on student: {:.4f}".format(
    f1_score(y_true = ['yes']*n_passed + ['no']*n_failed, y_pred =['yes']*n_students,pos_label=
            'yes',average='binary')))


F1 score for all yes on student: 0.8030


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [17]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Use 300 training points approx 75%
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
# Random state is 6 for shuffling 
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=6)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


In [18]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify=y_all, test_size=0.24, random_state=42)

print ("Grad rate of the training set: {:.2f}%".format(100 * (y_train == 'yes').mean()))
print ("Grad rate of the testing set: {:.2f}%".format(100 * (y_test == 'yes').mean()))

Grad rate of the training set: 67.00%
Grad rate of the testing set: 67.37%


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** Three supervised learning models

1 Decision Tree 

General Application: Classification, can be Apple vs Orange recognition problem

Strength: easy interpretation, intuitive attributes and properties, binary 1||0 thus doesn't need normalization, 

Weakness: subject to overfit

Reason: The output is easier to understand and more comprehensive finding the cause of student failure.


2 Support Vector Machine

General Application : Classification, facial expression classification. It's complex and has time constraint. Morelikely anything dealing with texture can be classified as support vector machine problem.

Strength: Could avoid overfit because it is more complex optimization problem

Weakness: Cubic computation time can be really sensitive in large number of data sets.

Reason: It will work well since the training and test set consider small data set size and it won't give so much noise. 


3 Naive Bayes

General Application: again Classification but also it can use as recognizing algorithm. One good example in classification is Spam email or not Spam.

Strength: Simple, quicker, need less training data

Weakness: It can't learn cross interaction between features. Need big data set in order to get decent estimation. Precision and recall will be really low. This can be a discussion in generative (GNB) vs discriminative model (Logistic Regression)

Reason: By taking a look at the strength, should give a shot to try.Conditional independece assumption might be good reason in this case. It is also part of the consideration since the training set and test set consider small and high bias classifier.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [46]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("------------------------------------------------------")
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')

#def train_training_sizes(clf,sizes):
    
#    for training_size in training_sizes:
#        rnd = np.random.choice(num_train, training_size, replace=False)
#        rnd_X_train = X_train.iloc[rnd, :]
#        rnd_y_train = y_train.iloc[rnd]
#        predict = train_predict(clf, rnd_X_train, rnd_y_train, X_test, y_test)
        
#    return predict


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print ("------------------------------------------------------")

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [47]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
# Overall the best model per F1 score test observation is SVC - clf_B
clf_A = DecisionTreeClassifier() #The best F1 score 200 test set, F1=0.7481
clf_B = SVC() #The best F1 Score 300 test set, F1 = 0.8456
clf_C = GaussianNB() # The best F1 Score 200 test set, F1=0.7826
# Try logistic regression as comparison with GNB
clf_D = LogisticRegression() #The best F1 score 300 test set , F1=0.8175

# loop thru models, then thru train sizes
for clf in [clf_A, clf_B, clf_C]:
    print ("\n{}: \n".format(clf.__class__.__name__))
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)

#training_sizes =[100,200,300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
#train_training_sizes(clf_A,training_sizes)


DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 100. . .
------------------------------------------------------
Trained model in 0.0040 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6829.
------------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 200. . .
------------------------------------------------------
Trained model in 0.0060 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7000.
------------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 300. . .
------------------------------------------------------
Trained model in 0.0070 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 

In [48]:
#train_training_sizes(clf_B,training_sizes)

In [49]:
#train_training_sizes(clf_C,training_sizes)

In [50]:
#train_training_sizes(clf_D,training_sizes)

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree** The timing slightly different because I had an issue with my Mac machine so I ran in Win 7  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0015          |     0.0003             |   1.0000         |    0.7302       |
| 200               |         0.0023          |     0.0003             |   1.0000         |    0.7717       |
| 300               |         0.0027          |     0.0003             |   1.0000         |    0.7258       |

** Classifer 2 - Support Vector Machine**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0026          |     0.0010             |   0.8354         |    0.8025       |
| 200               |         0.0045          |     0.0036             |   0.8431         |    0.8105       |
| 300               |         0.0098          |     0.0068             |   0.8664         |    0.8052       |

** Classifer 3 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0024          |     0.0007             |   0.7863         |    0.6723       |
| 200               |         0.0013          |     0.0005             |   0.7717         |    0.6992       |
| 300               |         0.0015          |     0.0009             |   0.7711         |    0.7402       |

** Classifer 4 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0024          |     0.0005             |   0.8657         |    0.7419       |
| 200               |         0.0023          |     0.0003             |   0.8483         |    0.7907       |
| 300               |         0.0038          |     0.0003             |   0.8268         |    0.8175       |


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Support Vector Machine. The reason is comparing from the table given, we can see that F1 score train, test and prediction time, cost and performance the SVM has greater chance of success in validating the model. Compared between the results in Tabular view, we can see that the SVM prediction will have the highest F1 scores over the test set. If the cost of using SVM is not a concern, this is the best option. This condition applies when it comes at the mid or end of school year (Around Christmas time, or Summer Holiday). I figure the computation relies on the speed of the computer. It can be fast IF the school principal or entity provides faster commputer to perform computational. However, based on the table we can say that it's relatively fast on the small data sets. Average training time will be 0.0056 seconds which relatively small. So let's simulate this. Assuming the total number of student will increase and say 2000 students, the average time will be 0.0056 and the test set will be increased let's say 100. Thus, (0.0056/100) * 2000 = 0.11 seconds. This is still considered fast.

**Answer: **

The way I do make prediction with SVC model is striking for high accuracy for the prediction. From the given data set, we can distinguish the main problem. Are the student pass or fail ? Then how do we approach this ? Classification is one method to solve the problem by drawing the lines between the feature space students who pass or fail. The goal is to make the prediction pass or fail by looking at the regression line.

1) The SVC takes data about past students (age,gender, family) and uses them to create function that draws a oundary between the students who pass stundent and fail student. Using SVM, we want observe the learning algorithm from the data provided on past students whether they passed or failed. This happens by looking at the lines of separation in the feature space between passed students versus failed students. The goal is to find the large difference between these two groups and create a prediction to determine the current student WILL pass or fail based on their data.

<img src='http://nlp.stanford.edu/IR-book/html/htmledition/img1260.png'>


2) Often, though its not easy to draw a decision boundary in low diensions, so the SVM separates the classes by translating the students data into dimensional space. We might want to look the boundary or linear separator in higher dimensional space. In this case, we are doing kernel method. Using this function created with the students we already know graduated or not, the SVM can look at new students data and predict by seeing which side of the separator data point falls on. 

<img src ='https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1464624531/1gvce.png'>

Now, we understand the problem then how the SVC can help to solve this issue. The idea of SVC is an alogrithm which is trained to find the separator (in this case, regression linear line is the separator between pass and fail tests).

The goal of the SVC is trying to maximize the margin width around linear line separation which it also means less overfit. Also from the graph given, we can see there are support vectors which right to the separator or closer to the margin. This is what defines the model. Once the pattern is recognized, it's not needed to keep the data points.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [76]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score

# TODO: Create the parameters list you wish to tune
#parameters = [
#  {'C': [1, 10, 100, 200, 300, 400, 500, 600, 700],
#   'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
#   'kernel': ['rbf'], 'tol':[1e-2, 1e-3, 1e-4, 1e-5]
#  }
#]
#def_gamma = 1 / n_features
parameters = [
  {'C': [1,10,100],
   #'gamma': [def_gamma / 10, def_gamma * 10, def_gamma, def_gamma * 100 ],
   'gamma':[0.03333333333333333],
   'kernel': ['rbf'], 
   'tol':[0.1],
   #'class_weight':['auto']
  }
]

# TODO: Initialize the classifier
clf = clf_B

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
clf_i = grid_obj.best_params_

print ("Best Params: ", clf_i)
# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

Best Params:  {'kernel': 'rbf', 'tol': 0.1, 'gamma': 0.03333333333333333, 'C': 1}
Made predictions in 0.0180 seconds.
Tuned model has a training F1 score of 0.9034.
Made predictions in 0.0060 seconds.
Tuned model has a testing F1 score of 0.8235.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The returned test set score for the tunned model finally higher than the untunned. Did couple research finding out the best gamma float for this and crossed out the assumption untunned model is better than tunned. The review clued me class imbalance and the cv parameter in gridsearch. In theory, the tunned model should always return better or at least the same as the untunned model. The final score for the tunned model is Training 0.9034 and Testing 0.8235. While the Untunned model is (the highest) training 0.8431 and testing (highest) 0.8105. The tunned model is better in training the model and slightly better in testing the model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.